# Hull White Model

In this notebook we demonstrate the use of the Hull White model.

Furthermore, we use our example implementation to analyse model properties.

## Model Setup and Analytic Formulas

We setup a Hull White model and demonstrate the use of basic analytic functions.

In [ ]:
import sys
sys.path.append('../') # make sure we can access the src/ folder

from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import brentq

from src.hull_white_model import HullWhiteModel
from src.monte_carlo_simulation import MonteCarloSimulation
from src.yieldcurve import YieldCurve

As first input we need to specify the initial yield curve.

In [ ]:
terms = [    '1y',    '2y',    '3y',    '4y',    '5y',    '6y',    '7y',    '8y',    '9y',   '10y',   '12y',   '15y',   '20y',   '25y',   '30y', '50y'   ] 
rates = [ 2.70e-2, 2.75e-2, 2.80e-2, 3.00e-2, 3.36e-2, 3.68e-2, 3.97e-2, 4.24e-2, 4.50e-2, 4.75e-2, 4.75e-2, 4.70e-2, 4.50e-2, 4.30e-2, 4.30e-2, 4.30e-2 ] 

disc_curve = YieldCurve(terms, rates)

Then we also specify the model parameters mean reversion and short rate volatility.

In [ ]:
mean_reversion = 0.03
volatility_times  = np.array([ 1.0, 2.0, 3.0, 5.0, 7.0, 10.0, ])
volatility_values = np.array([ 90., 80., 70., 60., 50., 40., ]) * 1e-4

Mean reversion is constant. Short rate volatility is a backward flat interpolated curve.

In [ ]:
plt.figure(figsize=(8,5))
plt.step(volatility_times, volatility_values, '*-')
plt.xlim((0.0, volatility_times[-1]))
plt.ylim((0.0, 0.01))
plt.xlabel(r'$t$')
plt.ylabel(r'$\sigma(t)$')


With yield curve, mean reversion, and piece-wise constant volatility parameter we can construct a Hull White model.

In [ ]:
model = HullWhiteModel(disc_curve, mean_reversion, volatility_times, volatility_values)

With the given model we can now apply our analytic functions in the Hull White model.

In [ ]:
t = 10.0
T = 20.0
x = np.linspace(-0.10, 0.10, 101)

zb = model.zero_bond(t, x, T)

plt.figure(figsize=(8,5))
plt.plot(x, zb)
plt.xlabel('state variable $x$')
plt.ylabel('zero coupon bond price $P(t,T)$')
plt.title('Zero coupon bond price for $t=10$ and $T=20$')

Similarly, we can calculate zero coupon bond options. Note that also for zero bond option put-call parity holds.

In [ ]:
strike_prices = np.linspace(0.4, 0.8, 101)

zb_call = np.array([model.zero_bond_option(t, T, K, call_or_put=+1) for K in strike_prices])
zb_put  = np.array([model.zero_bond_option(t, T, K, call_or_put=-1) for K in strike_prices])

fwd_bond = disc_curve.discount(T) / disc_curve.discount(t)
fwd_call = model.zero_bond_option(t, T, fwd_bond, call_or_put=+1)

plt.figure(figsize=(8,5))
plt.plot(strike_prices, zb_call, 'r-', label='call option')
plt.plot(strike_prices, zb_put,  'b-', label='put option')
plt.plot([fwd_bond], [fwd_call], 'go', label='put/call parity price')
plt.xlabel('strike price')
plt.ylabel('zero bond option price')
plt.legend()
plt.title('Zero bond option price for $t=10$ and $T=20$')

Also coupon bond options can be calculated.

In [ ]:
expiry_time = 10.0
pay_times  = np.linspace(11.0, 20.0, 10)
cash_flows = np.array([ 0.03 ] * len(pay_times))   # 3% annual coupon
cash_flows[-1] += 1.0  # final notional repayment

flows = pd.DataFrame()
flows['pay times']  = pay_times
flows['cash flows'] = cash_flows
display(flows)

strike_prices = np.linspace(0.7, 1.1, 101)
cb_call = np.array([ model.coupon_bond_option(expiry_time, pay_times, cash_flows, K, call_or_put=+1) for K in strike_prices ])
cb_put  = np.array([ model.coupon_bond_option(expiry_time, pay_times, cash_flows, K, call_or_put=-1) for K in strike_prices ])

plt.figure(figsize=(8,5))
plt.plot(strike_prices, cb_call, 'r-', label='call option')
plt.plot(strike_prices, cb_put,  'b-', label='put option')
plt.xlabel('strike price')
plt.ylabel('coupon bond option price')
plt.legend()
plt.title('Coupon bond option price')

## Impact of Volatility and Mean Reversion

In this section we analyse the impact of mean reversion and short rate volatility on model properties.

### Mean Reversion and Simulated State Variables

As a first step we want to analyse how mean reversion $a$ impacts paths of the state variable $x(t)$.

Mean reversion also impacts the overall variance of $x(t)$. We want to make observations for different mean reversion values comparable. To achieve this, we *calibrate* short rate volatility such that we have 100bp *flat volatility* in 5y and 10y. 



In [ ]:
def calibrated_model(curve, mean_reversion, T0, T1, flat_vol):
    def obj1(sigma1):
        model = HullWhiteModel(curve, mean_reversion, np.array([T0, T1]), np.array([sigma1, sigma1]))
        return model.variance(0.0,T0) - flat_vol**2 * T0
    sigma1 = brentq(obj1,1.0e-4,1.0e-1)
    #
    def obj2(sigma2):
        model = HullWhiteModel(curve, mean_reversion, np.array([T0, T1]), np.array([sigma1, sigma2]))
        return model.variance(0,T1) - flat_vol**2 * T1
    sigma2 = brentq(obj2,1.0e-4,1.0e-1)
    #
    return HullWhiteModel(curve, mean_reversion, np.array([T0, T1]), np.array([sigma1, sigma2]))

For such a calibrated model we can now analyse sampled paths.

In [ ]:
flat_curve = YieldCurve(['30y'],[0.03])

mean_reversions = [ -0.05, 1e-4, 0.05 ]
times = np.linspace(0.0, 10.0, 101)
n_paths = 10

fig, axs = plt.subplots(1, len(mean_reversions), sharey=True)
fig.set_size_inches(12, 5)
for a, ax in zip(mean_reversions,axs):
    model = calibrated_model(flat_curve, a, T0=5.0, T1=10.0, flat_vol=0.01)
    sim = MonteCarloSimulation(model, times, n_paths, seed=1234)
    for path in sim.X[:,0,:].T:
        ax.plot(times, path)
        ax.set_xlabel('simulation time $t$')
        ax.set_ylabel('state variable $x(t)$')
        ax.set_title('$a = %.2f$' % a)


Some runs of above simulation demonstrate, that for higher mean reversion we observe *more volatility* between 5y and 10y compared to lower mean reversion.

We can make above observation more clear by analysing forward volatility
$$
  \sigma_{\text{Fwd}}(T_0, T_1) =
  \sqrt{\frac{\text{Var}\left( x(T_1) | x(T_0) \right)}{T_1 - T_0}} =
  \sqrt{\frac{y(T_1) - G'(T_0,T_1)^2y(T_0)}{T_1 - T_0}}
$$
for given (fixed) values $y(T_0)$ and $y(T_1)$.

In [ ]:
flat_vol = 1.0e-2
T0 = 5.0
T1 = 10.0
y0 = flat_vol**2 * T0
y1 = flat_vol**2 * T1

sigma_fwd = lambda a : np.sqrt((y1 - np.exp(-a*(T1-T0))*y0)/(T1-T0))

mean_reversions = np.linspace(-0.1,0.1,101)

plt.figure(figsize=(8,5))
plt.plot(mean_reversions, sigma_fwd(mean_reversions)*1e+4)
plt.xlabel('mean reversion a')
plt.ylabel('forward volatility (bp)')
plt.title('T0 = 5y, T1 = 10y, spot sigma = 100bp')

### Modelled Yield Curves

We are particularly interested in which types of future yield curve can be modelled.

For that purpose we analyse forward rates $f(t,T)$ where observation time $t=5y$.

In [ ]:
t = 5.0
dT = np.linspace(0.0, 30.0, 301)
states = [ -0.10, -0.05, 0.0, 0.05, 0.1 ]
mean_reversions = [ -0.05, 1e-4, 0.05 ]

model = HullWhiteModel(disc_curve, a, np.array([0.0]), np.array([0.01]))

fig, axs = plt.subplots(1, len(mean_reversions))
fig.set_size_inches(12, 5)
for a, ax in zip(mean_reversions,axs):
    model = HullWhiteModel(disc_curve, a, np.array([0.0]), np.array([0.01]))
    ax.plot(0.0 + dT, [model.yield_curve.forwardRate(T) for T in 0.0+dT], label='$f(0,T)$' )
    for x in reversed(states):
        ax.plot(t + dT, [model.forward_rate(t, x, t+d) for d in dT], label='$x=%4.2f$' % x)
    ax.legend()
    ax.set_xlabel('maturity $T$')
    ax.set_ylabel('forward rate $f(t,T)$')
    ax.set_title('$a = %.2f$' % a)
plt.tight_layout()

## Model-implied Volatilities

Analysing and understanding model-implied volatilities is important because it shows which market prices can be matched with a model.

We consider normal model-implied volatilities. A normal model-implied volatility is obtained by inverting Bachelier's formula given a forward price of a European swaption that is calculated based on a Hull-White model.

In order to calculate model-implied volatilities we need the following steps:

  1. A European swaption reference instrument.

  2. A Hull-White model with initial yield curve, mean reversion and short rate volatilities.

  3. The representation of the European swaption as coupon bond option.

  4. Corresponding Hull-White model coupon bond option price, swaption annuity and forward swap rate.

  5. Inversion of Bachelier formula.

We illustrate it for a single European swaption first. W.l.o.g. we set projection curve equal to discount curve.

In [ ]:
from src.swaption import create_swaption

# swaption instrument
swaption = create_swaption('5y', '10y', disc_curve, disc_curve)

# model
mean_reversion = 0.03
volatility_times  = np.array([ 1.0, 2.0, 3.0, 5.0, 7.0, 10.0, ])
volatility_values = np.array([ 90., 80., 70., 60., 50., 40., ]) * 1e-4
model = HullWhiteModel(disc_curve, mean_reversion, volatility_times, volatility_values)

The bond option representation is provided by our swaption wrapper class.

In [ ]:
option = swaption.bond_option_details()
display(option)

This allows calculating the Hull-White model price.

In [ ]:
npv = model.coupon_bond_option(
    option['expiry_time'],
    option['pay_times'],
    option['cash_flows'],
    option['strike_price'],
    option['call_or_put']
)
display(npv)

Annuity, forward swap rate and option type can be obtained from the swaption instrument

In [ ]:
annuity = swaption.annuity()
fwd_swap_rate = swaption.fairRate()
call_or_put = swaption.call_or_put()
display(annuity, fwd_swap_rate, call_or_put)

Finally, we calculate the Normal implied volatility.

In [ ]:
from src.helpers import bachelier_implied_vol

implied_vol = bachelier_implied_vol(
    npv/annuity,
    swaption.fixed_rate(),
    fwd_swap_rate,
    option['expiry_time'],
    call_or_put,
    )
    
display(implied_vol*1e+4)

We get an implied volatility of about 60p for our example swaption and model.

In order to simplify our analysis we wrap implied volatility calculation into a function.

In [ ]:
def model_implied_volatility(model, expiry_term='10y', swap_term='10y', strike='ATM'):
    swaption = create_swaption(expiry_term, swap_term, model.yield_curve, model.yield_curve, strike)
    option = swaption.bond_option_details()
    fwd_price = model.coupon_bond_option(
        option['expiry_time'],
        option['pay_times'],
        option['cash_flows'],
        option['strike_price'],
        option['call_or_put']
        ) / swaption.annuity()
    implied_vol = bachelier_implied_vol(
        fwd_price,
        swaption.fixed_rate(),
        swaption.fairRate(),
        option['expiry_time'],
        swaption.call_or_put(),
        )
    return implied_vol


In [ ]:
model_implied_volatility(model, '5y', '10y')

### Volatility Skew and Smile

Volatility skew and smile is the behaviour of implied volatilities of an option for different strikes.

We pick a 10y (expiry) into 10y (swap term) swaption and analyse model-implied volatility smile for Hull White models with (constant) short rate volatility ranging from 50bp to 125bp and mean reversion ranging from -5% to 5%. 

In [ ]:
expiry_term = '10y'
swap_term = '10y'

swaption = create_swaption(expiry_term, swap_term, disc_curve, disc_curve)

Swaption strikes (i.e. the underlying swap fixed rate) is chosen relative to at-the-money.

In [ ]:
relative_strkes = np.linspace(-0.03, 0.03, 21)

fwd_rate = swaption.fairRate()
absolute_strikes = fwd_rate + relative_strkes

Now we can specify our volatility and mean reversion scenarios and plot implied volatilities.

In [ ]:
mean_reversions = [ -0.05, 1e-4, 0.05 ]
short_rate_vols = np.array([ 50, 75, 100, 125 ]) * 1e-4

fig, axs = plt.subplots(1, len(mean_reversions))
fig.set_size_inches(12, 5)
for a, ax in zip(mean_reversions,axs):
    for sigma in short_rate_vols:
        model = HullWhiteModel(disc_curve, a, np.array([0.0]), np.array([sigma]))
        vols = np.array([
            model_implied_volatility(model, expiry_term, swap_term, K) * 1e+4
            for K in absolute_strikes
        ])
        ax.plot(relative_strkes, vols, label='$\sigma=%.0f$bp' % (sigma*1e+4))
    ax.legend()
    ax.set_xlabel('relative strike')
    ax.set_ylabel('model-implied volatility (bp)')
    ax.set_title('$a=%.2f$' % a)
    ax.set_ylim((0, 250))
plt.tight_layout()

As a result we find that 

  - with a Hull White model we can only model flat volatility smile and

  - model-implied volatility decreases if mean reversion increases.

### At-the-money Volatility Surface

In this analysis step we focus on the following question: How does the shape of at-the-money volatility surface change if mean reversion changes.

European swaptions are represented by option expiry and swap term. Most relevant (and liquidly traded) options are at-the-money options where strike equals the forward swap rate.

An at-the-money Swaption volatility surface is spanned by the dimension expiry and swap term where the strike rate is always set equal to the corresponding forward swap rate.

Again, we want to make results for various mean reversion parameters comparable. Therefore we *fix* the 10y into 10y swaption model implied volatility at 100bp. And calibrate a constant short rate volatility to match the 10y into 10y swaption volatility.

In [ ]:
def model_from_swaption(swaption, mean_reversion):
    disc_curve = swaption.underlying_swap.discYieldCurve
    swaption_npv = swaption.npv()
    option = swaption.bond_option_details()
    def obj(sigma):
        model = HullWhiteModel(disc_curve, mean_reversion, np.array([0.0]), np.array([sigma]))
        model_price = model.coupon_bond_option(
            option['expiry_time'],
            option['pay_times'],
            option['cash_flows'],
            option['strike_price'],
            option['call_or_put']
            )
        return model_price - swaption_npv
    sigma = brentq(obj,1.0e-4,1.0e-1)
    return HullWhiteModel(disc_curve, mean_reversion, np.array([0.0]), np.array([sigma]))

We set a list of expiries and swap terms, calculate implied volatilities and plot results.

In [ ]:
swaption = create_swaption('10y', '10y', disc_curve, disc_curve, normalVolatility=0.01)

expiry_terms = np.arange(1, 21, 1)
swap_terms = np.arange(1, 21, 1)

mean_reversions = [ -0.05, 1e-4, 0.05 ]

fig, axs = plt.subplots(1, len(mean_reversions), subplot_kw=dict(projection='3d'))
fig.set_size_inches(12, 8)
for a, ax in zip(mean_reversions,axs):
    model = model_from_swaption(swaption, a)
    print('a = %.2f, sigma = %.4f' % (a, model.volatility_values[0]))
    vols = np.array([
        [ model_implied_volatility(model, str(e)+'y', str(s)+'y') for s in swap_terms]
        for e in expiry_terms
    ]) * 1e+4
    x, y = np.meshgrid(expiry_terms, swap_terms)
    ax.plot_surface(x, y, vols, cmap=cm.coolwarm, linewidth=0, antialiased=False)
    ax.set_xlim(0, 20)
    ax.set_ylim(0, 20)
    ax.set_zlim(50, 150)
    ax.set_xticks([0, 5, 10, 15, 20])
    ax.set_yticks([0, 5, 10, 15, 20])
    ax.set_xlabel('expiry terms (y)')
    ax.set_ylabel('swap terms (y)')
    ax.set_zlabel('model-implied normal volatility (bp)')
    ax.set_title('$a=%.2f$' % a)
plt.tight_layout()

We find that

  - mean reversion impacts at-the-money volatilities in expiry term and swap term dimension,
  - small/zero mean reversion yields a flat at-the-money volatility surface.